In [55]:
#STREAMS2TABLE
#written by Zachary J.J. Roper
#z.roper@vanderbilt.edu, zach@roper.io

#A script for converting raw event codes and timestamps into a trial-item dataset

In [56]:
#This notebook requires the following modules
import os
import numpy
import csv

In [57]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [58]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [59]:
events.count('2756') #catch trials

7

In [60]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [61]:
#uberlist

In [62]:
'2810' in uberlist[74][2]

False

In [63]:
RT = [None]*len(pairedlist)
RTbeg = 0*len(pairedlist)
RTend = 0*len(pairedlist)
for i, trial in enumerate(uberlist):
    #print(i,trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

In [64]:
#RT

In [65]:
#The following provides indexed lists for each of the individual event codes of interest 

In [66]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

In [67]:
#float(times[74])-float(times[70])

In [68]:
#Saccade

In [69]:
#for i, x in enumerate(Saccade):
#    print(Saccade[i],TargetOnset[i])

In [70]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [71]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [72]:
#print(TimeTrials[0])

In [73]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [74]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
correct = [None]*len(Trials)
tloc = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2756' in trial:
        catch[i] = 1
    else:
        catch[i] = 0
    if '888' in trial:
        correct[i] = 1
    else:
        correct[i] = 0
    #if '5000' in trial:
    #    tloc[i] = 360
    if '5045' in trial:
        tloc[i] = 45
    elif '5090' in trial:
        tloc[i] = 90
    elif '5135' in trial:
        tloc[i] = 135
    elif '5180' in trial:
        tloc[i] = 180
    elif '5225' in trial:
        tloc[i] = 225
    elif '5270' in trial:
        tloc[i] = 270
    elif '5315' in trial:
        tloc[i] = 315
    else:
        tloc[i] = 0

In [75]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [76]:
RT = [0]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '2810' in trial: #and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [77]:
#RT

In [78]:
checksum = sum(error)+sum(abort)+sum(catch)

In [79]:
checksum

329

In [80]:
path = os.getcwd()
csvfile = (path+"/behaviorArray.csv")
trial = 0
triallist = [None]*len(abort)
with open(csvfile, "w") as output:
    for i,x in enumerate(abort):
        trial += 1
        triallist[i] = trial 
        print(i+1,abort[i],accuracy[i],error[i],correct[i],catch[i],RT[i],tloc[i])
    header = ['Trial', 'Abort', 'Accuracy', 'Error', 'Correct', 'Catch', 'RT', 'TarLoc']    
    dataset = list(zip(triallist,abort,accuracy,error,correct,catch,RT,tloc))
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(dataset)

1 1 0 0 0 0 0 0
2 0 1 0 1 0 77.98790000000008 45
3 1 0 0 0 0 0 225
4 1 0 0 0 0 0 315
5 0 1 0 1 0 175.96409999999742 45
6 0 1 0 1 0 139.96020000000135 0
7 1 0 0 0 0 0 180
8 0 1 0 1 0 187.96540000000095 90
9 1 0 0 0 0 0 270
10 1 0 0 0 0 0 45
11 1 0 0 0 0 0 225
12 1 0 0 0 0 0 45
13 0 1 0 1 0 212.00890000000072 90
14 1 0 0 0 0 0 270
15 0 1 0 1 0 219.99619999999413 270
16 0 1 0 1 0 177.9710999999952 0
17 1 0 0 0 0 0 180
18 1 0 0 0 0 0 315
19 1 0 0 0 0 0 135
20 1 0 0 0 0 0 90
21 1 0 0 0 0 0 135
22 0 1 0 1 0 156.01660000000265 180
23 0 1 0 1 0 161.91479999999865 90
24 0 1 0 1 0 181.98510000000533 0
25 1 0 0 0 0 0 45
26 1 0 0 0 0 0 225
27 1 0 0 0 0 0 0
28 1 0 0 0 0 0 135
29 0 1 0 1 0 157.98280000000523 0
30 0 1 0 1 0 165.96979999999894 135
31 1 0 0 0 0 0 225
32 0 1 0 1 0 173.95709999999963 45
33 0 1 0 1 0 156.01659999999538 0
34 0 1 0 1 0 141.9673000000039 135
35 1 0 0 0 0 0 0
36 1 0 0 0 0 0 135
37 1 0 0 0 0 0 135
38 1 0 0 0 0 0 135
39 0 1 0 1 0 173.95709999999963 90
40 0 1 0 1 0 153.968600000

545 1 0 0 0 0 0 315
546 0 1 0 1 0 202.01470000005793 270
547 0 1 0 1 0 256.0 0
548 1 0 0 0 0 0 45
549 1 0 0 0 0 0 180
550 0 1 0 1 0 191.9796000000788 45
551 0 1 0 1 0 203.9806999999564 0
552 0 1 0 1 0 149.95449999999255 135
553 0 1 0 1 0 201.97369999997318 225
554 0 1 0 1 0 231.99750000005588 270
555 0 0 0 0 1 0 90
556 1 0 0 0 0 0 135
557 0 1 0 1 0 246.0056999999797 180
558 1 0 0 0 0 0 180
559 0 1 0 1 0 219.99620000005234 315
560 0 1 0 1 0 243.95779999997467 45
561 0 1 0 1 0 189.97239999996964 135
562 0 1 0 1 0 219.99600000004284 135
563 1 0 0 0 0 0 135
564 0 1 0 1 0 192.02040000003763 0
565 0 1 0 1 0 200.0079000000842 270
566 0 1 0 1 0 297.98420000006445 270
567 1 0 0 0 0 0 270
568 0 1 0 1 0 205.98790000006557 135
569 1 0 0 0 0 0 270
570 0 0 1 0 0 247.97200000006706 180
571 1 0 0 0 0 0 270
572 1 0 0 0 0 0 45
573 0 1 0 1 0 241.99179999995977 90
574 0 1 0 1 0 259.9732999999542 315
575 1 0 0 0 0 0 135
576 0 1 0 1 0 249.9788000001572 315
577 0 1 0 1 0 239.98479999997653 45
578 1 0 0 0 0 0

In [81]:
sum(catch)

7